In [1]:
def timer(func):
    def timer_wrap(*args, **kwargs):
        start = datetime.now()
        func(*args,**kwargs)
        stop = datetime.now()
        print( (stop-start).microseconds)
    return timer_wrap

In [2]:
import numpy as np
from datetime import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
I_ext = np.ones(shape=(2048,2048))
I_90 = np.ones(shape=(2048,2048))
I_135 = np.ones(shape=(2048,2048))
I_45 = np.ones(shape=(2048,2048))
I_0 = np.ones(shape=(2048,2048))

In [4]:
img_raw = np.stack((I_ext, I_0, I_45, I_90, I_135))

In [5]:
img_raw.shape

(5, 2048, 2048)

In [6]:
start = datetime.now()
img_raw_flat = np.reshape(img_raw, (5, 2048*2048))
stop = datetime.now()
print((stop-start).microseconds)

212


In [7]:
def compute_inst_matrix():
    chi = 0.2
    inst_mat = np.array([[1, 0, 0, -1],
                         [1, np.sin(chi), 0, -np.cos(chi)],
                         [1, 0, np.sin(chi), -np.cos(chi)],
                         [1, -np.sin(chi), 0, -np.cos(chi)],
                         [1, 0, -np.sin(chi), -np.cos(chi)]])

    inst_mat_inv = np.linalg.pinv(inst_mat)
    return inst_mat_inv

In [8]:
start = datetime.now()

inverse = compute_inst_matrix()

stop = datetime.now()
print((stop-start).microseconds)

5161


In [9]:
inverse.shape
inverse.strides
type(inverse[0][0])


(4, 5)

(40, 8)

numpy.float64

In [10]:
img_raw_flat.shape
img_raw_flat.strides
type(img_raw_flat[0][0])

(5, 4194304)

(33554432, 8)

numpy.float64

In [24]:
#============= timing test =====================
# data is float 54

start = datetime.now()
img_stokes_flat = np.dot(inverse, img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)

115470


In [12]:
inverse.flags
img_raw_flat.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [25]:
np.show_config()

mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAV

In [32]:
from scipy.linalg.blas import dgemm
from scipy.linalg.blas import dsymm

In [28]:
start = datetime.now()
img_stokes_flat = dgemm(alpha=1., a=inverse, b=img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)

230132


In [33]:
start = datetime.now()
img_stokes_flat = dsymm(alpha=1., a=inverse, b=img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)

233890


In [62]:
# =========== timing test ===================
# data is float 32

inverse = inverse.astype(np.float32)
img_raw_flat = img_raw_flat.astype(np.float32)

start = datetime.now()
img_stokes_flat = np.dot(inverse, img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)

29665


In [64]:
# =========== timing test ===================
# data is float 32

inverse = inverse.astype(np.float32)
img_raw_flat = img_raw_flat.astype(np.float32)

start = datetime.now()
img_stokes_flat = np.dot(inverse, img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)

26573


In [66]:
import numpy.distutils.system_info as sysinfo
sysinfo.get_info('blas')

/var/folders/43/m27gc01d3c15nl1v53qdv58c0000gp/T/tmpaqtyv4h_/source.c:1:10: fatal error: 'cblas.h' file not found
#include <cblas.h>
         ^~~~~~~~~
1 error generated.
/var/folders/43/m27gc01d3c15nl1v53qdv58c0000gp/T/tmpaqtyv4h_/source.c:1:10: fatal error: 'cblas.h' file not found
#include <cblas.h>
         ^~~~~~~~~
1 error generated.


{'libraries': ['blas', 'blas'],
 'library_dirs': ['/usr/lib'],
 'include_dirs': ['/usr/local/include',
  '/usr/include',
  '/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']}

In [68]:
np.__config__.show()

mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAV